# Entrenamiento Random Forest - Predicción de Severidad

## Objetivo
Entrenar un modelo de Machine Learning Supervisado (Random Forest) para predecir la severidad de emergencias médicas.

## Dataset
- **Archivo:** `emergencia_pacientes.csv`
- **Registros:** 2000 casos etiquetados
- **Target:** severidad (crítico, alto, medio, bajo)

## Arquitectura
Este notebook es para **ENTRENAMIENTO OFFLINE**. El modelo entrenado se usará en la capa `negocio/ml/`

In [1]:
# Importaciones
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
import joblib
import os

print('Librerías importadas correctamente')

Librerías importadas correctamente


## 1. Cargar datos

In [3]:
# Cargar CSV
ruta_csv = '../archivos_csv/emergencia_pacientes.csv'
df = pd.read_csv(ruta_csv)

print(f'Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas')
df.head()

Dataset cargado: 2000 filas, 16 columnas


,paciente_id,edad,sexo,presion_sistolica,presion_diastolica,frecuencia_cardiaca,frecuencia_respiratoria,temperatura,saturacion_oxigeno,tipo_incidente,nivel_dolor,tiempo_desde_incidente,tiene_seguro,timestamp,severidad,decision
0,PAC00001,52,F,146,74,79,30,37.1,105,problema_cardiaco,3,12,True,2024-12-03 17:28:32.464724,medio,traslado
1,PAC00002,93,F,117,56,120,12,36.4,99,problema_respiratorio,3,58,False,2025-02-23 17:28:32.464724,medio,traslado
2,PAC00003,15,F,116,73,74,18,38.9,96,quemadura,10,139,True,2025-02-25 17:28:32.464724,bajo,ambulatoria
3,PAC00004,72,F,107,76,69,21,37.0,97,problema_respiratorio,2,87,False,2024-12-23 17:28:32.464724,medio,traslado
4,PAC00005,61,F,114,85,86,23,34.4,99,fractura,10,107,True,2025-08-17 17:28:32.464724,medio,traslado


## 2. Análisis Exploratorio

In [4]:
# Información del dataset
print('=== INFORMACIÓN DEL DATASET ===')
print(df.info())

print('\n=== VALORES FALTANTES ===')
print(df.isnull().sum())

print('\n=== DISTRIBUCIÓN DE SEVERIDAD (TARGET) ===')
print(df['severidad'].value_counts())
print('\nProporción:')
print(df['severidad'].value_counts(normalize=True))

=== INFORMACIÓN DEL DATASET ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   paciente_id              2000 non-null   object 
 1   edad                     2000 non-null   int64  
 2   sexo                     2000 non-null   object 
 3   presion_sistolica        2000 non-null   int64  
 4   presion_diastolica       2000 non-null   int64  
 5   frecuencia_cardiaca      2000 non-null   int64  
 6   frecuencia_respiratoria  2000 non-null   int64  
 7   temperatura              2000 non-null   float64
 8   saturacion_oxigeno       2000 non-null   int64  
 9   tipo_incidente           2000 non-null   object 
 10  nivel_dolor              2000 non-null   int64  
 11  tiempo_desde_incidente   2000 non-null   int64  
 12  tiene_seguro             2000 non-null   bool   
 13  timestamp                2000 non-null   objec

## 3. Preprocesamiento

In [5]:
# Features (X) - signos vitales y datos del incidente
features = [
    'edad',
    'presion_sistolica',
    'presion_diastolica',
    'frecuencia_cardiaca',
    'frecuencia_respiratoria',
    'temperatura',
    'saturacion_oxigeno',
    'nivel_dolor',
    'tiempo_desde_incidente'
]

# Variables categóricas a codificar
le_sexo = LabelEncoder()
le_tipo_incidente = LabelEncoder()

df['sexo_encoded'] = le_sexo.fit_transform(df['sexo'])
df['tipo_incidente_encoded'] = le_tipo_incidente.fit_transform(df['tipo_incidente'])

# Agregar features codificadas
features.extend(['sexo_encoded', 'tipo_incidente_encoded'])

# Preparar X e y
X = df[features]
y = df['severidad']

print(f'Features seleccionadas: {len(features)}')
print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')

Features seleccionadas: 11
X shape: (2000, 11)
y shape: (2000,)


## 4. División train/test

In [6]:
# Split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Mantener proporción de clases
)

print(f'Train set: {X_train.shape[0]} muestras')
print(f'Test set: {X_test.shape[0]} muestras')

Train set: 1600 muestras
Test set: 400 muestras


## 5. Entrenamiento Random Forest

In [7]:
# Inicializar modelo
modelo_rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

# Entrenar
print('Entrenando Random Forest...')
modelo_rf.fit(X_train, y_train)
print('Entrenamiento completado!')

Entrenando Random Forest...
Entrenamiento completado!


## 6. Evaluación del modelo

In [8]:
# Predicciones
y_pred = modelo_rf.predict(X_test)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)')

# Classification Report
print('\n=== CLASSIFICATION REPORT ===')
print(classification_report(y_test, y_pred))

# Confusion Matrix
print('\n=== CONFUSION MATRIX ===')
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.9250 (92.50%)

=== CLASSIFICATION REPORT ===
              precision    recall  f1-score   support

        alto       0.91      0.74      0.81        65
        bajo       0.97      0.99      0.98       169
     critico       0.00      0.00      0.00         4
       medio       0.89      0.96      0.92       162

    accuracy                           0.93       400
   macro avg       0.69      0.67      0.68       400
weighted avg       0.92      0.93      0.92       400


=== CONFUSION MATRIX ===
[[ 48   0   0  17]
 [  0 167   0   2]
 [  3   0   0   1]
 [  2   5   0 155]]


D:\Semestre 2-2025\Sofware II\Segundo Parcial\segundoparcial\ServicioDecision\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Semestre 2-2025\Sofware II\Segundo Parcial\segundoparcial\ServicioDecision\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Semestre 2-2025\Sofware II\Segundo Parcial\segundoparcial\ServicioDecision\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no pre

## 7. Importancia de Features

In [9]:
# Feature importance
importancia = pd.DataFrame({
    'feature': features,
    'importance': modelo_rf.feature_importances_
}).sort_values('importance', ascending=False)

print('=== IMPORTANCIA DE FEATURES ===')
print(importancia)

=== IMPORTANCIA DE FEATURES ===
                    feature  importance
10   tipo_incidente_encoded    0.301592
5               temperatura    0.244256
3       frecuencia_cardiaca    0.112886
1         presion_sistolica    0.108769
0                      edad    0.074662
7               nivel_dolor    0.056078
8    tiempo_desde_incidente    0.027604
2        presion_diastolica    0.026173
6        saturacion_oxigeno    0.025290
4   frecuencia_respiratoria    0.018626
9              sexo_encoded    0.004066


## 8. Guardar modelo y encoders

In [10]:
# Crear directorio si no existe
os.makedirs('../modelos_ml', exist_ok=True)

# Guardar modelo
joblib.dump(modelo_rf, '../modelos_ml/modelo_severidad.pkl')
print('Modelo guardado: modelos_ml/modelo_severidad.pkl')

# Guardar encoders
joblib.dump(le_sexo, '../modelos_ml/encoder_sexo.pkl')
joblib.dump(le_tipo_incidente, '../modelos_ml/encoder_tipo_incidente.pkl')
print('Encoders guardados')

# Guardar lista de features
joblib.dump(features, '../modelos_ml/features_list.pkl')
print('Features guardadas')

print('\n¡Entrenamiento completado exitosamente!')

Modelo guardado: modelos_ml/modelo_severidad.pkl
Encoders guardados
Features guardadas

¡Entrenamiento completado exitosamente!


## 9. Prueba de predicción

In [11]:
# Caso de prueba
paciente_prueba = {
    'edad': 65,
    'presion_sistolica': 180,
    'presion_diastolica': 100,
    'frecuencia_cardiaca': 110,
    'frecuencia_respiratoria': 25,
    'temperatura': 38.5,
    'saturacion_oxigeno': 88,
    'nivel_dolor': 9,
    'tiempo_desde_incidente': 15,
    'sexo': 'M',
    'tipo_incidente': 'cardiovascular'
}

# Codificar categóricas
paciente_prueba['sexo_encoded'] = le_sexo.transform([paciente_prueba['sexo']])[0]
paciente_prueba['tipo_incidente_encoded'] = le_tipo_incidente.transform([paciente_prueba['tipo_incidente']])[0]

# Preparar datos
X_prueba = pd.DataFrame([paciente_prueba])[features]

# Predecir
severidad_predicha = modelo_rf.predict(X_prueba)[0]
probabilidades = modelo_rf.predict_proba(X_prueba)[0]

print('=== PREDICCIÓN DE PRUEBA ===')
print(f'Severidad predicha: {severidad_predicha}')
print(f'\nProbabilidades por clase:')
for clase, prob in zip(modelo_rf.classes_, probabilidades):
    print(f'  {clase}: {prob:.4f} ({prob*100:.2f}%)')

ValueError: y contains previously unseen labels: 'cardiovascular'